In [28]:
%%file useful_structure.py
from copy import deepcopy


class PartialLockedDict:
    """
    THIS IS A BASECLASS
    
    PartialLockedDict conceptually consists one built-in dict and one outer dict.
    The built-in dict is considered constant and users are not suggested to change it.
    The outer dict is defined by the user and is subjected to all kinds of changes.
    PartialLockedDict provides methods accessing data with high security.
    
    
    
    METHODS
    ========
    these are all classmethods
    
    _error_change_builtin(cls, key):
        throws KeyError based on the input key.
        Call this function when built-in dict is going to get changed.
    
    _get_all(cls):
        * currently not implemented *
        return the dict covering both built-in dict and outer dict
        OUTPUT: * should be * dict (reference)
        
    _get_builtin(cls):
        * currently not implemented *
        return the built-in dict
        OUTPUT: * should be * dict (reference)
    
    reset(cls):
        * currently not implemented *
        clear the outer dict
        OUTPUT: * should be * cls
    
    getcopy(cls, key):
        retrieve the value with key in the whole dict
        INPUTS:  key, dict-key-like, the key to the value of interest
        OUTPUTS: value, dict-value-like, the value retrieved
    
    getcopy_all(cls):
        returns a deepcopy of the dict covering both built-in dict and outer dict
        OUTPUTS: dict (deepcopy)
        
    getcopy_builtin(cls):
        returns a deepcopy of the built-in dict
        OUTPUTS: dict (deepcopy)
        
    update(cls, key, value):
        updates the outer dict's dict-item indexed key with value
        if key in built-in dict, calls cls._error_change_builtin(cls, key) to raise KeyError
        INPUTS:  key, dict-key-like, the key to the item to be updated
                 value, dict-value-like, the value of update
        OUTPUTS: cls
    
    update_group(cls, dict_update):
        updates the outer dict with dict_update
        if dict_update contains any key that is also in the built-in dict, 
        calls cls._error_change_builtin(cls, key) to raise KeyError
        INPUTS:  dict_update, dict, the group of items as the update
        OUTPUTS: cls
        
    remove(cls, key):
        remove the key indexed dict-item in the outer dict
        if key in built-in dict, calls cls._error_change_builtin(cls, key) to raise KeyError
        INPUTS:  key, dict-key-like, the key to the item to be removed
        OUTPUTS: cls
    
    """
    @classmethod
    def _error_change_builtin(cls, key):
        raise KeyError
        
    @classmethod
    def _get_all(cls):
        raise NotImplementedError
        
    @classmethod
    def _get_builtin(cls):
        raise NotImplementedError
        
    @classmethod
    def reset(cls):
        raise NotImplementedError
        
    @classmethod
    def getcopy(cls, key):
        return cls._get_all()[key]
    
    @classmethod
    def getcopy_all(cls):
        return deepcopy(cls._get_all())
    
    @classmethod
    def getcopy_builtin(cls):
        return deepcopy(cls._get_builtin())
    
    @classmethod
    def update(cls, key, value):
        if key in cls._get_builtin():
            cls._error_change_builtin(key)
        cls._get_all()[key] = value
        return cls
    
    @classmethod
    def update_group(cls, dict_update):
        for key in dict_update.keys():
            if key in cls._get_builtin():
                cls._error_change_builtin(key)
        for key,value in dict_update.items():
            cls._get_all()[key] = value
        return cls
    
    @classmethod
    def remove(cls, key):
        if key in cls._get_builtin():
            cls._error_change_builtin(key)
        del cls._get_all()[key]
        return cls

Overwriting useful_structure.py


In [23]:
%%file check_and_response.py
class ValueCheck:
    """
    ValueCheck provides formatted response to invalid values.
    
    
    ATTRIBUTES
    ===========
    
    _criterion:  function-like, criterion to decide the validity of some value.
        ------------------------------
        should get called as self._criterion(x):
        INPUTS:  x, numeric type, to-be-judged value
        OUTPUTS: j, boolean, judgement
        
    _name: name of the criterion. will be used in the response(...) method.
    
    
    METHOD
    =======
    
    response(self, x, label, term): response to some value if it passes self._criterion
        INPUTS: x, numeric type, to-be-judged value
                label, str, the math symbel of x
                term, str, the terminology of x
        NOTE:   get reponse when self._criterion(x) == True
                the response will be a ValueError with message formatted:
                [MESSAGE] - (label of x) = (x): (criterion name) (term of x) is prohibited
    
    
    INITIALIZATION
    ===============
    __init__(self, criterion, name)
    
    INPUTS: 
    -----------
        criterion: function-like, initializes _criterion
        name:      str, initializes _name
    
    
    EXAMPLE
    ========
    
    >>> try:
    ...     ValueCheck(lambda x:x>0, 'positive').response(1, 'x', 'integer')
    ... except ValueError as err:
    ...     print(err)
    x = 1.0000000000000000e+00: positive integer is prohibited.
    
    """
    
    def __init__(self, criterion, name):
        # criteron should be a function type
        # criteron(x) returns boolean value
        self._criterion = criterion
        self._name = name
        
    def response(self, x, label, term):
        if self._criterion(x):
            raise ValueError(
                '{} = {:18.16e}: {} {} is prohibited.' \
                .format(label, x, self._name, term))

Overwriting check_and_response.py


In [34]:
%%file mathematical_science.py
from copy import deepcopy


class MathModel:
    """
    THIS IS A BASE CLASS
    
    MathModel is basically a well wrapped function. 
    as a "model", it has hypothesis - some implicit parameters, and some math relationships.
    it can take in some inputs, check their validity, and compute some result. 
    
    MathModel should be able to check the validity of implicit params and provide the math relation 
    without instantiation. But to check the validity of model inputs, it may need information from 
    the instance itself.
    
    * following specifications are vulnerable to further inheritance *
    * these are just an example implementation on the common sense of a MathModel *
    * users are not demanded to follow these implementations, but might find them useful *
    
    
    ATTRIBUTES
    ===========
    
    _coeffparams: the implicit parameters of the model, initialized by the __init__ method.
    
    
    METHODS
    ========
    
    compute(self, check=True, **stateparams): compute the result on some inputs.
        INPUTS:  check, boolean, defaults True, whether to start a param check before computation.
                 stateparams, undefined non-positional, the inputs of the model.
        OUTPUTS: result, undefined, the output of the model.
        NOTE:    calls check_stateparams(...) to check inputs
                 calls _kernel(...) to do computation
                 
    get_coeffparams(self): return the implicit params, in a dict (deepcopy)
        
    check_stateparams(self, **stateparams): check validity of the model inputs.
        defaults doing nothing. user are expected to specify it in its inheritors.
        INPUTS:  stateparams, undefined non-positional, model inputs
        OUTPUTS: defaults nothing, but is expected to raise error if params are invalid
    
    check_coeffparams(**coeffparams): staticmethod, check validity of the model implicit params.
        defaults doing nothing. user are expected to specify it in its inheritors.
        INPUTS:  coeffparams, undefined non-positional, implicit params (hypothesis) of the model
        OUTPUTS: defaults nothing, but is expected to raise error if params are invalid
        
    _kernel(**params): staticmethod, the mathematical function.
        computes the result base on both model implicit params and model inputs.
        is the CORE of MathModel - the whole MathModel class is basically a wrapper of this function.
        will get called by the compute(...) method.
        INPUTS:  params, undefined non-positional, combination of implicit params and model inputs.
        OUTPUTS: result, undefined, the output of the model
        NOTE:    is the most efficient implementation one can write on the mathematical relation.
                 is expected to take in as less keyword arguments as possible.
                 is expected to do only computation. no checks should be involved.
                 is not suggested to get called from outside of the class for its vulnerablility to
                 invalid implicit params or invalid model inputs. 
    
    
    INITIALIZATION
    ===============
    __init__(self, check=True, **coeffparams)
    * this is just an example of initialization, with some unversality *
    * details could vary a lot for different models *
    
    INPUTS: 
    -----------
        check:       boolean, if True, check_stateparams(...) will get called to check coeffparams
        coeffparams: implicit params of the model, initializes self._coeffparams
        
        
    EXAMPLE
    ========

    >>> class somelaw(MathModel):
    ...     @staticmethod
    ...     def _kernel(T, E, R):
    ...         return E / (T * R)
    ...     @staticmethod
    ...     def check_coeffparams(R, **other_params):
    ...         if R == 0.0: raise ValueError
    ...     def check_stateparams(self, T, **other_params):
    ...         if T == 0.0: raise ValueError
    >>> somelaw(E=8.314, R=8.314).compute(T=2.0)
    0.5
    
    """
    def __init__(self, check=True, **coeffparams):
        if check:
            self.check_coeffparams(**coeffparams)
        self._coeffparams = coeffparams
        
    def compute(self, check=True, **stateparams):
        if check: 
            self.check_stateparams(**stateparams)
        return self._kernel(**self._coeffparams, **stateparams)
    
    def get_coeffparams(self):
        return deepcopy(self._coeffparams)
    
    def check_stateparams(self, **stateparams):
        pass
    
    @staticmethod
    def check_coeffparams(**coeffparams):
        pass
    
    @staticmethod
    def _kernel(**params):
        raise NotImplementedError

Overwriting mathematical_science.py


In [31]:
%%file CoeffLaw.py
from mathematical_science import MathModel
from check_and_response import ValueCheck
import numpy as np


class Constant(MathModel):
    """
    Constant reaction rate coefficient, inherited from MathModel
    its implicit param is k, which is valid when positive
    it does not take any model input, and the output is just k itself, 
    so check_stateparams(...) method need not be specified.
    
    
    ATTRIBUTES
    ===========
    _check_np: ValueCheck type, will reponse to non-positive input
    _k:        float, constant reaction rate coefficient
    
    other attributes follow the MathModel pattern, 
    including: _coeffparams
    
    
    METHODS
    ========
    compute(self, **other_params): compute constant reaction rate coefficient
        doesn't (even need to) call _kernel method, just returns self._k
        NOTE: non-positional arg, other_params, are placed in case too many inputs are passed.
              this notation is effective to the end of this file.
    check_coeffparams(k, **other_params): check if k is positive, raise ValueError if not.
        NOTE: calls _check_np.reponse(k)
    _kernel(k, **other_params): mathematical relation, returns k itself.
    
    other methods follow the MathModel pattern, 
    including: check_stateparams(...), get_coeffparams(...)
    
    
    
    INITIALIZATION
    ===============
    follows the MathModel pattern
    
    
    
    EXAMPLE
    ========
    >>> Constant(1.0).compute()
    1.0
    
    
    """
    
    _check_np = ValueCheck(lambda x:x<=0.0, 'non-positive')
    
    def __init__(self, check=True, k=1.0, **other_params):
        if check:
            self.check_coeffparams(k)
        self._k = k
        self._coeffparams = dict(k=k)
        
    def compute(self, check=True, **other_params):
        return self._k
    
    @staticmethod
    def check_coeffparams(k, **other_params):
        Constant._check_np.response(k, 'k', 'reaction rate coefficient')
        
    @staticmethod
    def _kernel(k, **other_params):
        return k


# ================================================================================ #
    
class Arrhenius(MathModel):
    """
    Arrhenius reaction rate coefficient, inherited from MathModel
    
    implicit params, all float:
        A, Arrhenius prefactor, valid when positive, defaults 1.0
        E, reaction energy,     valid as always,     defaults 0.0
        R, ideal gas constant,  valid when positive, defaults 8.314
    model input, all float:
        T, temperature,         valid when positive, defaults 1e-16
        
    by default this model will behave like Constant
    
    
    ATTRIBUTES
    ===========
    _check_np:  ValueCheck type, will reponse to non-positive input
    _A, _E, _R: implicit params, as specified above
    
    other attributes follow the MathModel pattern, 
    including: _coeffparams
    
    
    METHODS
    ========
    compute(self, check=True, T=1e-16, **other_params): 
        compute Arrhenius reaction rate coefficient. follows the MathModel pattern.
    check_coeffparams(A, R, **other_params): 
        check if A, R are positive, raise ValueError if not.
        NOTE: calls _check_np.reponse() on A and R
    check_stateparams(self, T, **other_params):
        check if T is positive, raise ValueError if not
    _kernel(k, **other_params): mathematical relation.
    
    other methods follow the MathModel pattern, 
    including: get_coeffparams(...)
    
    
    INITIALIZATION
    ===============
    follows the MathModel pattern
    
    
    EXAMPLE
    ========
    >>> Arrhenius(A=np.e, E=8.314).compute(T=1.0)
    1.0
    
    """
    
    _check_np = ValueCheck(lambda x:x<=0.0, 'non-positive')
    
    def __init__(self, check=True, A=1.0, E=0.0, R=8.314, **other_params):
        if check:
            self.check_coeffparams(A, R)
        self._A, self._E, self._R = A, E, R
        self._coeffparams = dict(A=A, E=E, R=R)
        
    def compute(self, check=True, T=1e-16, **other_params):
        if check: 
            self.check_stateparams(T)
        return self._kernel(T, self._A, self._E, self._R)
    
    def check_stateparams(self, T, **other_params):
        self._check_np.response(T, 'T', 'temperature')
        
    @staticmethod
    def check_coeffparams(A, R, **other_params):
        Arrhenius._check_np.response(R, 'R', 'ideal gas constant')
        Arrhenius._check_np.response(A, 'A', 'Arrhenius prefactor')
        
    @staticmethod
    def _kernel(T, A, E, R, **other_params):
        return A * (np.e ** (-E / (R * T)))

    
    
# ================================================================================ #
    
class modArrhenius(MathModel):
    """
    Modified Arrhenius reaction rate coefficient, inherited from MathModel
    
    implicit params, all float:
        A, Arrhenius prefactor,          valid when positive, defaults 1.0
        b, modified Arrhenius parameter, valid when real,     defaults 0.0
        E, reaction energy,              valid as always,     defaults 0.0
        R, ideal gas constant,           valid when positive, defaults 8.314
    model input, all float:
        T, temperature,                  valid when positive, defaults 1e-16
        
    by default this model will behave like Constant
    if only b follows the default value, this model will behave like Arrhenius
    
    other attributes follow the MathModel pattern, 
    including: _coeffparams
    
    
    ATTRIBUTES
    ===========
    _check_np:      ValueCheck type, will reponse to non-positive input
    _A, _b, _E, _R: implicit params, as specified above
    
    
    METHODS
    ========
    compute(self, check=True, T=1e-16, **other_params): 
        compute Arrhenius reaction rate coefficient. follows the MathModel pattern.
    check_coeffparams(A, R, **other_params): 
        check if A, R are positive, raise ValueError if not.
        NOTE: calls _check_np.reponse() on A and R
    check_stateparams(self, T, **other_params):
        check if T is positive, raise ValueError if not
    _kernel(k, **other_params): mathematical relation.
    
    other attributes follow the MathModel pattern, 
    including: _coeffparams
    
    
    INITIALIZATION
    ===============
    follows the MathModel pattern
    
    
    EXAMPLE
    ========
    >>> modArrhenius(A=np.e, b=-1.0, E=4.157).compute(T=0.5)
    2.0
    """
    
    _check_np = ValueCheck(lambda x:x<=0.0, 'non-positive')
    
    def __init__(self, check=True, A=1.0, b=0.0, E=0.0, R=8.314, **other_params):
        if check:
            self.check_coeffparams(A, R)
        self._A, self._b, self._E, self._R = A, b, E, R
        self._coeffparams = dict(A=A, b=b, E=E, R=R)
        
    def compute(self, check=True, T=1e-16, **other_params):
        if check: 
            self.check_stateparams(T)
        return self._kernel(T, self._A, self._b, self._E, self._R)
    
    def check_stateparams(self, T, **other_params):
        self._check_np.response(T, 'T', 'temperature')
        
    @staticmethod
    def check_coeffparams(A, R, **other_params):
        modArrhenius._check_np.response(A, 'A', 'Arrhenius prefactor')
        modArrhenius._check_np.response(R, 'R', 'ideal gas constant')
        
    @staticmethod
    def _kernel(T, A, b, E, R, **other_params):
        return A * (T ** b) * (np.e ** (-E / (R * T)))

Overwriting CoeffLaw.py


In [49]:
%%file Reaction.py
from useful_structure import PartialLockedDict
from mathematical_science import MathModel
from copy import deepcopy
import numpy as np
import CoeffLaw


class Reaction:
    """
    Reaction keeps all the infomation from one given reaction. It also helps to select 
    the right law function, as attribute rateCoeff, to compute the reaction rate coefficient 
    based on the classification of the given reaction. In the inner class _CoeffLawDict several 
    laws including constant coeffs, Arrhenius coeffs and Modified Arrhenius coeffs have been 
    implemented, and _CoeffLawDict forms up a dict-like structure to manage them.
    
    
    
    ATTRIBUTES
    ===========
    
    _params: dict, records the parameters has has built up the instance.
        it is initialized by __init__ and can be set to other values using set_params method.
        the keys of _params are fixed to the following list and should not be changed:
        [KEYS] - reversible, TYPE, ID, coeffLaw, coeffParams, coeffUnits, reactants, products
    
    rateCoeff: function, typically called as self.rateCoeff(check=True, **state)
        this function compute the reaction rate coefficient under certain state.
        it is selected from the _CoeffLawDict by __init__, and reseting self._params should 
        automatically respecify this rateCoeff. 
        --------------------------------------------
        For function rateCoeff(self, check=True, **state):
        INPUTS: 
            check, boolean, decides whether to check the state input, defaults True
            state, non-positional args, currently known as potentially containing:
                'T': float, temperature under which reaction happens, valid when positive
        OUTPUTS: k, float, reaction rate coefficient
        
    _CoeffLawDict: inner class, dict-like structure
        _CoeffLawDict Keeps and manages the law functions (MathModel type, see CoeffLaw.py) 
        that might be used to compute the reaction rate coefficients. Each function is 
        associated with a key - mostly their name, in a string. It also provides several 
        dict-like methods. All these will be further specified inside the _CoeffLawDict.
    
    
    
    METHODS
    ========
    
    get_params(self): 
        return a deepcopy of self._params
        OUTPUTS: self._params, dict (deepcopy)
        
    set_params(self, **kwargs):
        update self._params with kwargs.
        only keys that are originally in _params would be updated.
        will call self._check_params() to see if this update is valid.
        if 'coeffLaw' is updated, will call self._specify_rateCoeff() to reset self.rateCoeff
        INPUTS:  kwargs, non-positional, contains the updates 
        OUTPUTS: self, Reaction instance
        
    getReactants(self): 
        returns the reactants in a dict
        OUTPUTS: self._params['reactants'], dict (deepcopy)
            has the form of (reactant name):(stoich coeff)
        
    getProducts(self): 
        returns the reactants in a dict
        OUTPUTS: self._params['products'], dict (deepcopy)
            has the form of (product name):(stoich coeff)
        
    _check_params(self):
        check if self._params are valid.
        will get called by the __init__ method.
        it raises `NotImplementedError` if:
            self._params['reversible'] == True
            self._params['TYPE']       != 'Elementary'
            self._params['coeffLaw'] not in self._CoeffLawDict._dict_all
        it raises `ValueError` if:
            any stoich coeffs in self._params['reactants'] and self._params['products']
            are either non-integer or negative integer
        NOTE: _check_params does not check if self._params['coeffParams'] are valid.
            if wanted, the check_coeffparams method of the referred MathModel type should be
            called seperately from this _check_params. see _specify_rateCoeff below.
            
    _specify_rateCoeff(self):
        select the right law (MathModel type) to compute reaction rate coefficients, 
        and initialize it with self._params['coeffParams']. 
        self will get changed - attribute rateCoeff will get updated.
        will get called by the __init__ method.
        OUTPUTS: self.rateCoeff, function
            self.rateCoef(...) itself does not need self._params['coeffParams'] as inputs
        NOTE: _specify_rateCoeff does not check the validity of self._params['coeffParams']
            explicitly. However, it will call the __init__ method of a MathModel type, which 
            is expected to call self.check_coeffparams(...) to check the inputing coeffParams 
            by default. If the coeffParams are invalid, a ValueError is expected to get raised 
            from inside that check_coeffparams method.
            
    __repr__(self):
        return a wrapped dict of all params, namely str(self._params)
        OUTPUTS: representational str, valid input for eval()
        
    __str__(self):
        return a str to show the contents of self. 
        when printed out, there will be two parts:
            the reaction equation, in a chemistry convention
            the params list, in a 'param name: param value' fashion
        OUTPUTS: descriptive str
        
    
    
    INITIALIZATION
    ===============  
    
    INPUTS:
    ---------------------
    
        ID:          str keyword, defaults 'reaction', reaction id

        reversible:  boolean keyword, defaults False, reversibility
                     if True, will raise NotImplementedError

        TYPE:        str keyword, defaults 'Elementary', reaction type, 
                     if not 'Elementary', will raise NotImplementedError

        coeffLaw:    str keyword, defaults 'Constant', 
                     name of the law that computes the reaction rate coefficients
                     if not in Reaction._CoeffLawDict._dict_all, will raise NotImplementedError

        coeffParams: dict keyword, defaults {}, param values required by coeffLaw
                     if not empty, should be in form of paramName(str): paramValue(float)
                     should not contain state params such as temperature and concentration
                     coeffParams will be checked based on the input of coeffLaw,
                     if such check cannot pass, ValueError will be raised

        coeffUnits:  dict keyword, defaults {}, param units associated with coeffParams
                     if not empty, should be in form of paramName(str): paramUnit(str)

        reactants:   dict keyword, defaults {}, reactants
                     if not empty, should be in form of reactantName(str): stoichCoeff(int)
                     non-int or negative stoich. coeffs will raise ValueError

        products:    dict keyword, defaults {}, products
                     if not empty, should be in form of productName(str): stoichCoeff(int)
                     non-int or negative stoich. coeffs will raise ValueError

        kwargs:      some non-positional arguments that are currently not helpful
        
        
    NOTES:
    ---------------------
        attributes _params and rateCoeff will get initialized.
        inner class _CoeffLawDict does not require instantiation.
        several argument checks will run automatically:
            --------------------------------------------------------------------------------
            reversible           if True                                 NotImplementedError
            TYPE                 if not 'Elementary'                     NotImplementedError
            coeffLaw             if not in _CoeffLawDict._dict_all       NotImplementedError
            coeffParams          if cannot pass the param check          ValueError
                                 provided by the claimed coeffLaw
            reactants, products  if non-integer or negative integer      ValueError
                                 stoich coeffs detected
            --------------------------------------------------------------------------------
            
            
    
    EXAMPLES
    =========
    
    >>> r = Reaction( \
            reactants=dict(H=1,O2=1), \
            products=dict(OH=1,H=1), \
            coeffLaw='Arrhenius', \
            coeffParams=dict(A=np.e, E=8.314)\
        )
    >>> eval(repr(r))['coeffLaw']
    'Arrhenius'
    >>> print(str(r))
    ========================================
    Reaction Equation:
    1H + 1O2 [=] 1OH + 1H
    ----------------------------------------
    Reaction Info:
    reversible: False
    TYPE: Elementary
    ID: reaction
    coeffLaw: Arrhenius
    coeffParams: {'A': 2.718281828459045, 'E': 8.314, 'R': 8.314}
    coeffUnits: {}
    reactants: {'H': 1, 'O2': 1}
    products: {'OH': 1, 'H': 1}
    ========================================
    >>> r.rateCoeff(T=1.0)
    1.0
    >>> r.getReactants()
    {'H': 1, 'O2': 1}
    >>> r.set_params(reactants=dict(H=2,O2=2)).getReactants()
    {'H': 2, 'O2': 2}
    
    """
    
    class _CoeffLawDict(PartialLockedDict):
        """
        _CoeffLawDict keeps and manages the built-in laws (CoeffLaw inheritors) that compute 
        the reaction rate coefficients. At the same time it also allow user to add their self
        -defined laws with similar interface - the simplest way is just making a subclass from the 
        CoeffLaw class, and specifying the _kernel staticmethod.
        
        _CoeffLawDict inherits the PartialLockedDict class.
        
        
        ATTRIBUTES
        ===========
        
        built-in laws (CoeffLaw inheritors) includes:
            `Constant`     for constant coeffs
            `Arrhenius`    for Arrhenius coeffs
            `modArrhenius` for Modified Arrhenius coeffs
            
        _dict_builtin: dict
            the built-in dict which is not supposed to be changed.
            records mapping relation from names to the associated CoeffLaw inheritors,
            such as: ` 'Constant': CoeffLaw.Constant `
            
        _dict_all: dict
            a dict that contains what's in the built-in dict as well as user defined mappings.
            all searches and updates would be made on this dict.
            
            
        METHODS
        ========
        all methods are classmethods inherited from PartialLockedDict, among which the 
        following methods get further specification in this class:
        
        _error_change_builtin(cls, key):
            specified the message thrown by the KeyError
            input key will be clarified in that message
            
        _get_all(cls): 
            return _CoeffLawDict._dict_all, as a reference
            OUTPUTS: _dict_all, dict (reference)
            
        _get_builtin(cls):
            return _CoeffLawDict._dict_builtin, as a reference
            OUTPUTS: _dict_builtin, dict (reference)
            
        other methods stays the same as in the PartialLockedDict class
            
            
        EXAMPLES
        =========
        >>> class somelaw(MathModel):
        ...     @staticmethod
        ...     def _kernel(T, A, **other_params):
        ...         return T * A
        >>> Reaction._CoeffLawDict.update('sl', somelaw).getcopy_all().keys()
        dict_keys(['Constant', 'Arrhenius', 'modArrhenius', 'sl'])
        >>> r = Reaction( \
            reactants=dict(H=1,O2=1), \
            products=dict(OH=1,H=1), \
            coeffLaw='sl', \
            coeffParams=dict(A=2.0)\
        )
        >>> r.rateCoeff(T=0.1)
        0.2
        >>> Reaction._CoeffLawDict.reset().getcopy_all().keys()
        dict_keys(['Constant', 'Arrhenius', 'modArrhenius'])
        """
            
        _dict_builtin = {
            'Constant'    :CoeffLaw.Constant, 
            'Arrhenius'   :CoeffLaw.Arrhenius, 
            'modArrhenius':CoeffLaw.modArrhenius
        }
        _dict_all = deepcopy(_dict_builtin) 
        
        @classmethod
        def _error_change_builtin(cls, key):
            raise KeyError(' '.join([
                'LawName = {}'.format(key),
                'exists as a built-in law.',
                'Changing a built-in law is prohibited.']))
        @classmethod
        def _get_all(cls):
            return cls._dict_all
        @classmethod
        def _get_builtin(cls):
            return cls._dict_builtin
        @classmethod
        def reset(cls):
            cls._dict_all = deepcopy(cls._dict_builtin)
            return cls
            
    
    def __init__(
        self, 
        reversible  = False, 
        TYPE        = 'Elementary', 
        ID          = 'reaction', 
        coeffLaw    = 'Constant', 
        coeffParams = {},
        coeffUnits  = {},
        reactants   = {},
        products    = {},
        **kwargs
    ):
        self._params = {
            'reversible'  : reversible, 
            'TYPE'        : TYPE, 
            'ID'          : ID, 
            'coeffLaw'    : coeffLaw, 
            'coeffParams' : coeffParams,
            'coeffUnits'  : coeffUnits,
            'reactants'   : reactants,
            'products'    : products
        }
        self._check_params()
        self._specify_rateCoeff(coeffLaw, coeffParams)
        # this brings up the attribute self.rateCoeff
    
    def get_params(self):
        return deepcopy(self._params)
    
    def set_params(self, **kwargs):
        old_params = deepcopy(self._params)
        kwargs_eliminated = deepcopy(kwargs)
        for k in kwargs.keys():
            if k not in self._params:
                kwargs_eliminated.pop(k)
        self._params.update(**kwargs_eliminated)
        if self._params != old_params:
            self._check_params()
            coeffLaw = self._params['coeffLaw']
            coeffParams = self._params['coeffParams']
            if (old_params['coeffLaw'] != coeffLaw \
                or old_params['coeffParams'] != coeffParams):
                self._specify_rateCoeff(coeffLaw, coeffParams)
        return self
    
    def getReactants(self):
        return self._params['reactants']
    
    def getProducts(self):
        return self._params['products']
    
    def _check_params(self):
        if self._params['reversible']:
            raise NotImplementedError(
                'Reversible reaction is not implemented.')
        if self._params['TYPE'] != 'Elementary':
            raise NotImplementedError(' '.join([
                'TYPE = {}.'.format(self._params['TYPE']),
                'Non-elementary reaction is not implemented.']))
        if not self._params['coeffLaw'] in self._CoeffLawDict._dict_all:
            raise NotImplementedError(' '.join([
                'coeffLaw = {}.'.format(self._params['coeffLaw']),
                'Refered reaction rate coefficient law is not implemented.']))
        for ele,stoich in self._params['reactants'].items():
            if type(stoich) != int or stoich < 0:
                raise ValueError(' '.join([
                    'Reactant {}:{}.'.format(ele, stoich),
                    'Stoich. coeff must be a non-negative integer.']))
        for ele,stoich in self._params['products'].items():
            if type(stoich) != int or stoich < 0:
                raise ValueError(' '.join([
                    'Product {}:{}.'.format(ele, stoich),
                    'Stoich. coeff must be a non-negative integer.']))
                
    def _specify_rateCoeff(self, coeffLaw, coeffParams):
        selection = self._CoeffLawDict._dict_all[coeffLaw](**coeffParams)
        self._params['coeffParams'] = selection.get_coeffparams()
        def rateCoeff_specified(check=True, **stateparams):
            return selection.compute(check, **stateparams)
        self.rateCoeff = rateCoeff_specified
        return self.rateCoeff
    
    def __repr__(self):
        return str(self._params)
    
    def __str__(self):
        streq_lefthand = ' + '.join(
            ['{}{}'.format(v,k) for k,v in self._params['reactants'].items()])
        streq_righthand = ' + '.join(
            ['{}{}'.format(v,k) for k,v in self._params['products'].items()])
        streq_full = ' [=] '.join([streq_lefthand, streq_righthand])
        strparams = '\n'.join(
            [': '.join([str(k), str(v)]) for k,v in self._params.items()])
        return '\n'.join([
            '=' * 40, 'Reaction Equation:', streq_full, 
            '-' * 40, 'Reaction Info:', strparams,
            '=' * 40])

Overwriting Reaction.py


In [50]:
!pytest --doctest-modules

============================= test session starts ==============================
platform darwin -- Python 3.6.1, pytest-3.0.7, py-1.4.33, pluggy-0.4.0
rootdir: /Users/shell/Documents/GitHub/cs207-FinalProject, inifile: setup.cfg
plugins: cov-2.5.1
collected 15 items 

CoeffLaw.py ...
Reaction.py ..
check_and_response.py .
mathematical_science.py .
test_Reaction.py ........Coverage.py warning: Module roots was never imported. (module-not-imported)
Coverage.py warning: No data was collected. (no-data-collected)


========================== 15 passed in 0.25 seconds ===========================
ERROR: Failed to generate report: No data to report.



In [51]:
!pytest --cov --cov-report term-missing

============================= test session starts ==============================
platform darwin -- Python 3.6.1, pytest-3.0.7, py-1.4.33, pluggy-0.4.0
rootdir: /Users/shell/Documents/GitHub/cs207-FinalProject, inifile: setup.cfg
plugins: cov-2.5.1
collected 15 items 

CoeffLaw.py ...
Reaction.py ..
check_and_response.py .
mathematical_science.py .
test_Reaction.py ........

---------- coverage: platform darwin, python 3.6.1-final-0 -----------
Name                      Stmts   Miss  Cover   Missing
-------------------------------------------------------
CoeffLaw.py                  52      1    98%   69
Reaction.py                  70      0   100%
check_and_response.py         7      0   100%
mathematical_science.py      18      1    94%   112
test_Reaction.py            102      0   100%
useful_structure.py          33      4    88%   74, 78, 82, 86
-------------------------------------------------------
TOTAL                       282      6    98%


========================== 15 p

In [1]:
from Reaction import Reaction
import numpy as np
r = Reaction(
    reactants=dict(H=1,O2=1), products=dict(OH=1,H=1),
    coeffLaw='modArrhenius', coeffParams=dict(b=1.0, E=8.314)
)
print(str(r))
print(r.rateCoeff(check=False, T=-1.0))
print(r.getReactants())

Reaction Equation:
1H + 1O2 [=] 1OH + 1H
----------------------------------------
Reaction Info:
reversible: False
TYPE: Elementary
ID: reaction
coeffLaw: modArrhenius
coeffParams: {'b': 1.0, 'E': 8.314}
coeffUnits: {}
reactants: {'H': 1, 'O2': 1}
products: {'OH': 1, 'H': 1}
-2.718281828459045
{'H': 1, 'O2': 1}


In [2]:
try:
    Reaction._CoeffLawDict.reset()
    _r = Reaction(coeffLaw='_', coeffParams=dict(A=0.1))
except NotImplementedError as err:
    print(err)

coeffLaw = _. Refered reaction rate coefficient law is not implemented.


In [3]:
from mathematical_science import MathModel
class _law(MathModel):
    @staticmethod
    def _kernel(T, A, **other_params):
        return T * A

Reaction._CoeffLawDict.update('_', _law)
_r = Reaction(coeffLaw='_', coeffParams=dict(A=0.1))
_r.rateCoeff(T=2.0)

0.2